In [2]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
import os
import numpy as np
import logging
import tqdm
from sklearn.metrics import average_precision_score, roc_auc_score
import os
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator
import tqdm
from typing import Protocol

import h2o
from sklearn.tree import DecisionTreeClassifier

h2o.init(verbose=False)

In [3]:
outputs_df = pd.read_parquet("outputs_openai_embeddings_v1.parquet")

In [5]:
outputs_df.test_set.value_counts()

test_set
False    73
True     40
Name: count, dtype: int64

In [5]:
class Classifier(Protocol):
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """Fit the model to the training data."""
        
    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict labels for the given data."""
        return np.array([])

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Predict probability estimates for the given data."""
        return np.array([])


class H2O(BaseEstimator):
    def __init__(self, **kwargs):
        h2o.connect(verbose=False)
        self.clf = H2OAutoML(**kwargs)
        # self._CustomH2O__input = self._H2OAutoML__input
        # super().__init__(**kwargs)
        
    # def __init__(self):
    #     # Initialize your H2O model here
    #     pass
    def fit(self, X: np.ndarray, y: np.ndarray):
        h2o.connect(verbose=False)
        
        X_df = pd.DataFrame(X).add_prefix("X_")
        self.X_columns = X_df.columns.to_list()
        y_df = pd.DataFrame(y).add_prefix("y_").astype("int")
                                        
        y_col = y_df.columns[0]
        train_h2o_df = h2o.H2OFrame(X_df.join(y_df))
        train_h2o_df[y_col] = train_h2o_df[y_col].asfactor()
        self.clf.train(x=self.X_columns, y=y_df.columns[0], training_frame=train_h2o_df)

    def predict(self, X: np.ndarray) -> np.ndarray:
        h2o.connect(verbose=False)
        # Use your trained H2O model to make predictions on the given features (X)
        prediction =  self.clf.predict(h2o.H2OFrame(X, column_names=self.X_columns))
        if prediction is None:
            raise ValueError("Prediction is None")
        return prediction.as_data_frame()["predict"].to_numpy()
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        h2o.connect(verbose=False)
        # Use your trained H2O model to make predictions on the given features (X)
        prediction =  self.clf.predict(h2o.H2OFrame(X, column_names=self.X_columns))
        if prediction is None:
            raise ValueError("Prediction is None")
        return prediction.as_data_frame()["p1"].to_numpy()

    def save(self, path: str):
        leader_clf = self.clf.leader
        if leader_clf:
            leader_clf.save_mojo(path)
    
    def load(self, path: str):
        self.clf = h2o.import_mojo(path)
    
    
class H2OMultiLabel:
    def __init__(self, **kwargs):
        h2o.connect(verbose=False)
        self.clfs = [H2O(**kwargs) for _ in range(20)]
        # self._CustomH2O__input = self._H2OAutoML__input
        # super().__init__(**kwargs)
        
    def fit(self, X: np.ndarray, y: np.ndarray):
        h2o.connect(verbose=False)
        
        X_df = pd.DataFrame(X).add_prefix("X_")
        self.X_columns = X_df.columns.to_list()
        y_df = pd.DataFrame(y).add_prefix("y_").astype("int")
        # train individual models
        for i, clf in tqdm.tqdm(enumerate(self.clfs), total=len(self.clfs)):           
            y_col = y_df.columns[i]
            train_h2o_df = h2o.H2OFrame(X_df.join(y_df))
            train_h2o_df[y_col] = train_h2o_df[y_col].asfactor()
            clf.fit(X, y[:, i])

    def predict(self, X: np.ndarray) -> np.ndarray:
        h2o.connect(verbose=False)

        predictions = [clf.predict(X) for clf in self.clfs]
        # return np.concatenate(predictions, axis=1)
        return np.stack(predictions).T
        # # Use your trained H2O model to make predictions on the given features (X)
        # # prediction =  self.clf.predict(h2o.H2OFrame(X, column_names=self.X_columns))
        # if prediction is None:
        #     raise ValueError("Prediction is None")
        # return prediction.as_data_frame()["predict"].to_numpy()
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        h2o.connect(verbose=False)
        predictions_proba = [clf.predict_proba(X) for clf in self.clfs]
        # return np.concatenate(predictions_proba, axis=1)
        return np.stack(predictions_proba).T
        # Use your trained H2O model to make predictions on the given features (X)
        # prediction =  self.clf.predict(h2o.H2OFrame(X, column_names=self.X_columns))
        # if prediction is None:
        #     raise ValueError("Prediction is None")
        # return prediction.as_data_frame()["p1"].to_numpy()    
        
    def save(self, path):
        for i, clf in enumerate(self.clfs):
            clf.save(os.path.join(path, f"model_{i}.zip"))
            
    def load(self, path):
        self.clfs = [h2o.import_mojo(os.path.join(path, f"model_{i}.zip")) for i, _ in enumerate(self.clfs)]
            

In [43]:
outputs_df = pd.read_parquet("../data/processed/outputs_openai_embeddings_v1.parquet")
ppas = outputs_df.PPAs_list.explode().drop_duplicates().sort_values().to_list()
train_df = outputs_df[~outputs_df.test_set]
test_df = outputs_df[outputs_df.test_set]
X_labels = outputs_df.columns[outputs_df.columns.str.contains("openai_embedding_small")]
# y_label = "PPAs_list"
oversampling = True

X_train = train_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train = train_df[ppas].astype(int).to_numpy()

X_test = test_df[X_labels].to_numpy()
# y_test = test_df[y_label].to_numpy()
y_test = test_df[ppas].astype(int).to_numpy()


ros = RandomOverSampler(random_state=42)
train_oversamples_df, _ = ros.fit_resample(train_df, y=train_df["primary_ppa"])
X_train_oversampled = train_oversamples_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train_oversampled = train_oversamples_df[ppas].astype(int).to_numpy() 


def ppa_hierarchy(ppas_list: pd.Series):
    return ppas_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_numpy()


def experiment(name: str, clf: Classifier, X_train: np.ndarray, y_train: np.ndarray, 
               X_test: np.ndarray, y_test_df: pd.DataFrame, classes: list[str]):
    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_test)
    y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=classes)

    experiment = []
    for ppa in ppas:
        y_test_ppa = y_test_df[ppa]
        experiment.append({
            "name": name,
            "ppa": ppa,
            "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
            "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa]),
        })
    return experiment

class CustomDecisionTree(DecisionTreeClassifier):
    def predict_proba(self, X):
        probs = super().predict_proba(X)
        return np.array(probs)[:, :, 0].T

In [45]:
clf = H2OMultiLabel(max_models=20, seed=1)
name = "H2O MultiLabel"

all_experiments = []

# experiments = list(product([True, False], [True, False], clfs))
oversampling = True

train_df_exp = train_oversamples_df if oversampling else train_df

X_train_exp = X_train_oversampled if oversampling else X_train
y_train_exp = y_train_oversampled if oversampling else y_train

X_test_exp = X_test
y_test_exp_df = test_df[ppas].astype("int")

try:
    experiment_result = experiment(name, clf, X_train_exp, y_train_exp, X_test_exp, y_test_exp_df, ppas)
    all_experiments.extend(experiment_result)
except Exception as e:
    logging.error(f"Error in {((clf, name))}: {repr(e)}")

clf.save("model_20")

experiments_df = pd.DataFrame(all_experiments)
experiments_df.groupby("name")[["roc_auc", "average_precision"]].agg(["mean","std"]).sort_values(("roc_auc", "mean"), ascending=False)

  0%|          | 0/1 [00:00<?, ?it/s]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:34:17.151: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:34:54.256: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:35:32.620: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:36:09.416: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


...

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |

/Users/jm/Code/fao-ppa-classifier/.conda/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
100%|██████████| 1/1 [12:00<00:00, 720.91s/it]

███████████████████████████████████████████████████████| (done) 100%


roc_auc           average_precision          
                    mean       std              mean       std
name                                                          
H2O MultiLabel  0.886712  0.140054          0.589834  0.298051